<h2 style="color:green; font-size: 40px;" align="center">DATA ANALYSIS WITH SQL<h2>
<h2 style="color:grey" align="center">Setup up tables <h2>

```sql

-- Create the "CUSTOMER" table in "raw.source_1" schema
CREATE OR REPLACE TABLE RAW.SOURCE_1.CUSTOMER (
    C_CUSTKEY NUMBER,                -- Customer key (numeric)
    C_NAME STRING,                   -- Customer name (string)
    C_ADDRESS STRING,                -- Customer address (string)
    C_NATIONKEY NUMBER,              -- Nation key (numeric)
    C_PHONE STRING,                  -- Phone number (string)
    C_ACCTBAL NUMBER(18, 2),         -- Account balance (numeric with 2 decimal places)
    C_MKTSEGMENT STRING,             -- Market segment (string)
    C_COMMENT STRING                 -- Comment (string)
);

-- Create the ORDERS table
CREATE OR REPLACE TABLE raw.source_1.orders (
    O_ORDERKEY NUMBER,                -- Order key (numeric)
    O_CUSTKEY NUMBER,                 -- Customer key (numeric)
    O_ORDERSTATUS STRING,             -- Order status (string)
    O_TOTALPRICE NUMBER(18, 2),       -- Total price (numeric with decimal)
    O_ORDERDATE DATE,                 -- Order date (date)
    O_ORDERPRIORITY STRING,           -- Order priority (string)
    O_CLERK STRING,                   -- Clerk (string)
    O_SHIPPRIORITY NUMBER,            -- Shipping priority (numeric)
    O_COMMENT STRING                  -- Comment (string)
);

-- Create the NATION table
CREATE OR REPLACE TABLE raw.source_1.nation (
    N_NATIONKEY NUMBER,               -- Nation key (numeric)
    N_NAME STRING,                    -- Nation name (string)
    N_REGIONKEY NUMBER,               -- Region key (numeric)
    N_COMMENT STRING                  -- Comment (string)
);

-- You can skip this part if not familiar with AWS S3. Proceed and load the files into the table using the Snowflake Interface
-- CSV files are attached

-- Insert sample data into the CUSTOMER table
COPY INTO RAW.SOURCE_1.customer
FROM 's3://to_be_defined/file_name.csv'
FILE_FORMAT = csv_format;


-- Load data into the ORDERS table
COPY INTO raw.source_1.orders
FROM 's3://to_be_defined/orders.csv'
FILE_FORMAT = csv_format;


-- Load data into the NATION table
COPY INTO raw.source_1.nation
FROM 's3://to_be_defined/nation.csv'
FILE_FORMAT = csv_format;


<h2 style="color:grey" align="center"> Joning tables to start analysis <h2>

```sql

SELECT O_ORDERKEY, C_NAME, C_MKTSEGMENT, N.N_NAME, O_ORDERSTATUS, O_TOTALPRICE, O_ORDERDATE, O_ORDERPRIORITY  FROM RAW.SOURCE_1.ORDERS O
LEFT JOIN RAW.SOURCE_1.CUSTOMER C ON O.O_CUSTKEY = C.C_CUSTKEY 
LEFT JOIN RAW.SOURCE_1.NATION N ON C.C_NATIONKEY = N.N_NATIONKEY

![Imagen not found](https://drive.google.com/uc?export=view&id=1dZsSjczaSk5_cOgpMK19DV1mCrXxNJhE)

<h2 style="color:grey" align="center"> Lets start the Analysis <h2>
<h3 style="color:grey" align="center"> We are going to answer the folowing questions: <h3>

```sql

-- WHAT IS THE TOTAL VALUE OF ORDERS BY ORDER STATUS IN EACH COUNTRY?
SELECT 
    N_NAME, 
    CASE 
        WHEN O_ORDERSTATUS = 'F' THEN 'FINISHED'
        WHEN O_ORDERSTATUS = 'O' THEN 'ORDERED'
        WHEN O_ORDERSTATUS = 'P' THEN 'PENDING'
        ELSE O_ORDERSTATUS
    END AS ORDER_STATUS,
    TO_CHAR(SUM(O_TOTALPRICE), '$999,999,999') AS TOTAL_VALUE
FROM RAW.SOURCE_1.ORDERS O
LEFT JOIN RAW.SOURCE_1.CUSTOMER C ON O.O_CUSTKEY = C.C_CUSTKEY 
LEFT JOIN RAW.SOURCE_1.NATION N ON C.C_NATIONKEY = N.N_NATIONKEY
GROUP BY N_NAME, O_ORDERSTATUS 
ORDER BY N_NAME ASC;


![Imagen not found](https://drive.google.com/uc?export=view&id=12al3FgtQLklSHPWwpdaw0C4t9yLLXt0j)



```sql

-- WHAT IS THE AVERAGE PRICE OF ORDERS FOR EACH MARKET SEGMENT BY COUNTRY?
SELECT C_MKTSEGMENT, N_NAME, TO_CHAR(AVG(O_TOTALPRICE), '$999,999,999') AS AVERAGE_PRICE
FROM RAW.SOURCE_1.ORDERS O
LEFT JOIN RAW.SOURCE_1.CUSTOMER C ON O.O_CUSTKEY = C.C_CUSTKEY
LEFT JOIN RAW.SOURCE_1.NATION N ON C.C_NATIONKEY = N.N_NATIONKEY
GROUP BY C_MKTSEGMENT,N_NAME 
ORDER BY C_MKTSEGMENT ASC, AVERAGE_PRICE DESC ;



![Imagen not found](https://drive.google.com/uc?export=view&id=1qvIKubBFpSr4MyAoxwTtOMP_uOb7p6kp)


```sql
-- HOW IS THE NUMBER OF ORDERS DISTRIBUTED BY YEAR AND COUNTRY?
SELECT N_NAME, EXTRACT(YEAR FROM O_ORDERDATE) AS ORDER_YEAR, COUNT(*) AS NUMBER_OF_ORDERS
FROM RAW.SOURCE_1.ORDERS O
LEFT JOIN RAW.SOURCE_1.CUSTOMER C ON O.O_CUSTKEY = C.C_CUSTKEY 
LEFT JOIN RAW.SOURCE_1.NATION N ON C.C_NATIONKEY = N.N_NATIONKEY
GROUP BY N_NAME, EXTRACT(YEAR FROM O_ORDERDATE) 
ORDER BY N_NAME ASC, ORDER_YEAR DESC;




![Imagen not found](https://drive.google.com/uc?export=view&id=1ppn9hi-kM9uVUkrboQ0w-itGccK8XdGX)


```sql
-- WHAT IS THE MOST EXPENSIVE AND THE CHEAPEST ORDER IN EACH COUNTRY?
WITH PRICEEXTREMES AS (
    SELECT N_NAME, 
           TO_CHAR(MAX(O_TOTALPRICE), '$999,999,999') AS MAX_PRICE,  
            TO_CHAR(MIN(O_TOTALPRICE),'$999,999,999') AS MIN_PRICE
    FROM RAW.SOURCE_1.ORDERS O
    LEFT JOIN RAW.SOURCE_1.CUSTOMER C ON O.O_CUSTKEY = C.C_CUSTKEY 
    LEFT JOIN RAW.SOURCE_1.NATION N ON C.C_NATIONKEY = N.N_NATIONKEY
    GROUP BY N_NAME
)
SELECT N_NAME, MAX_PRICE, MIN_PRICE
FROM PRICEEXTREMES
ORDER BY N_NAME ASC;



![Imagen not found](https://drive.google.com/uc?export=view&id=18bDO0TC5Z0mst-d6UBA03A3YaZ8T3pKo)


```sql
-- RANKING OF CUSTOMERS BY TOTAL SPENDING IN EACH COUNTRY.
WITH CUSTOMERSPENDING AS (
    SELECT N_NAME, C_NAME, TO_CHAR(SUM(O_TOTALPRICE),'$999,999,999') AS TOTAL_SPENT
    FROM RAW.SOURCE_1.ORDERS O
    LEFT JOIN RAW.SOURCE_1.CUSTOMER C ON O.O_CUSTKEY = C.C_CUSTKEY 
    LEFT JOIN RAW.SOURCE_1.NATION N ON C.C_NATIONKEY = N.N_NATIONKEY
    GROUP BY N_NAME, C_NAME
)
SELECT N_NAME, C_NAME, TOTAL_SPENT,
       RANK() OVER (PARTITION BY N_NAME ORDER BY TOTAL_SPENT DESC) AS SPENDING_RANK
FROM CUSTOMERSPENDING;




![Imagen not found](https://drive.google.com/uc?export=154z5QpqpVQjT3k2cQI4P35qIgWT-Zhfn)



```sql
-- USE OF LAG TO COMPARE THE CURRENT ORDER PRICE WITH THE PREVIOUS ORDER PRICE OF EACH CUSTOMER:
WITH ORDERDETAILS AS( 
    SELECT 
    C_NAME, 
    O_ORDERDATE, 
    TO_CHAR(O_TOTALPRICE,'$999,999,999') AS CURRENT_ORDER,
    TO_CHAR(LAG(O_TOTALPRICE) OVER (PARTITION BY C_NAME ORDER BY O_ORDERDATE DESC),'$999,999,999') AS PREVIOUS_ORDER
FROM RAW.SOURCE_1.ORDERS O
LEFT JOIN RAW.SOURCE_1.CUSTOMER C ON O.O_CUSTKEY = C.C_CUSTKEY)
SELECT * FROM ORDERDETAILS
WHERE PREVIOUS_ORDER IS NOT NULL
ORDER BY O_ORDERDATE DESC;



![Imagen not found](https://drive.google.com/uc?export=view&id=1eN2ack2FovQZmjsnEXqwpWTdReV1mteg)



```sql
-- USE OF TOP 3 ORDERS BY TOTAL VALUE IN EACH COUNTRY AND YEAR:
SELECT 
    N_NAME, 
    EXTRACT(YEAR FROM O_ORDERDATE) AS YEAR,
    TO_CHAR(O_TOTALPRICE,'$999,999,999'),
    ROW_NUMBER() OVER (PARTITION BY N_NAME, EXTRACT(YEAR FROM O_ORDERDATE) ORDER BY O_TOTALPRICE DESC) AS RANK
FROM RAW.SOURCE_1.ORDERS O
LEFT JOIN RAW.SOURCE_1.CUSTOMER C ON O.O_CUSTKEY = C.C_CUSTKEY 
LEFT JOIN RAW.SOURCE_1.NATION N ON C.C_NATIONKEY = N.N_NATIONKEY
QUALIFY RANK IN (1,2,3);


![Imagen not found](https://drive.google.com/uc?export=view&id=1wUJi8o1iJnCqaV7B3OTDMBsVGLXisQFI)

